In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
import requests

url = "https://chm.iiserb.ac.in/faculty"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
html = response.text

print(html[:500])

<!--// Set XSS protection (legacy)-->

<style>
.MultiCarousel {
	float: left;
	overflow: hidden;
	padding: 15px;
	width: 100%;
	position: relative;
}
.MultiCarousel .MultiCarousel-inner {
	transition: 1s ease all;
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item {
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item > div {
	text-align: center;
	padding: 10px;
	margin: 10px;
	background: #EFF0F5;
	color: #E4E4E4;
}
.MultiCarousel .leftLst,
.MultiCarousel .r


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://chm.iiserb.ac.in"
FACULTY_PAGE = f"{BASE_URL}/faculty"   # ✅ Correct URL

headers = {"User-Agent": "Mozilla/5.0"}

# Fetch the main page
response = requests.get(FACULTY_PAGE, headers=headers, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

faculty_blocks = soup.select("div.col-md-3.prdct-grid")

with open("iiserbchem_fac.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Name", "Designation", "Email", "Phone", "Image", "Profile Link", "PhD", "Postdoc"])

    for block in faculty_blocks:
        # Extract image
        img_tag = block.find("img")
        image = BASE_URL + "/" + img_tag["src"] if img_tag else ""

        # Name & Profile link
        a_tag = block.select_one("div.product-name a[rel='author']")
        name = a_tag.text.strip() if a_tag else ""
        rel_link = a_tag.get("href") if a_tag else ""
        profile_link = BASE_URL + "/" + rel_link if rel_link else ""

        # Other details
        p_tag = block.select_one("div.product-name p")
        designation = email = phone = ""
        if p_tag:
            lines = [l.strip() for l in p_tag.get_text(separator="\n").split("\n") if l.strip()]
            if len(lines) > 0: designation = lines[0]
            if len(lines) > 1: email = lines[1]
            if len(lines) > 2: phone = lines[2]

        # Initialize additional fields
        phd = postdoc = ""

        # Scrape individual profile page for PhD/Postdoc info
        if profile_link:
            try:
                prof_resp = requests.get(profile_link, headers=headers, verify=False)
                prof_resp.raise_for_status()
                prof_soup = BeautifulSoup(prof_resp.text, "html.parser")
                li_tags = prof_soup.select("div.row.clearfix li")
                for li in li_tags:
                    text = li.get_text(strip=True)
                    if "Ph.D" in text or "PhD" in text:
                        phd = text
                    elif "Postdoctoral" in text or "Postdoc" in text:
                        postdoc = text
                time.sleep(0.3)
            except Exception as e:
                print(f"⚠️ Failed to fetch {profile_link}: {e}")

        writer.writerow([name, designation, email, phone, image, profile_link, phd, postdoc])

print("✅ Scraping completed! Data saved to iiserb_faculty.csv")


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/p

✅ Scraping completed! Data saved to iiserb_faculty.csv
